In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad
import dipy.reconst.cross_validation as xval
import dipy.data as dpd
import utils
import model as mm 
from model import Model, BiExponentialIsotropicModel
from parallelization import parallelization

In [2]:
sphere = dpd.get_sphere()
sphere = sphere.subdivide(1)

In [3]:
# Load Data
data_dict = utils.read_data()
data = data_dict['seen']['signal'][:, 0]
bvals = data_dict['seen']['bvals']
bvecs = data_dict['seen']['bvecs']
delta = data_dict['seen']['delta']
Delta = data_dict['seen']['Delta']
te = data_dict['seen']['TE']
g = data_dict['seen']['g']

In [4]:
# Split Data 66% seen and 34% unseen. B0 are always seen, unseen is one shell per TE quadrupel
n = 4
data_seen = []
data_unseen = []
te_seen = []
te_unseen = []
gtab_seen = []
gtab_unseen = []

for j in range(0, n): 
    mask_seen, mask_unseen = utils.split_data()
    data_seen.append(data[mask_seen])
    data_unseen.append(data[mask_unseen])
    te_seen.append(te[mask_seen])
    te_unseen.append(te[mask_unseen])
    bvals_seen_temp = bvals[mask_seen]
    bvecs_seen_temp = bvecs[mask_seen]
    delta_seen_temp = delta[mask_seen]
    Delta_seen_temp = Delta[mask_seen]
    gtab_seen.append(grad.gradient_table(bvals_seen_temp, bvecs_seen_temp, big_delta=Delta_seen_temp, small_delta=delta_seen_temp))
    bvals_unseen_temp = bvals[mask_unseen]
    bvecs_unseen_temp = bvecs[mask_unseen]
    delta_unseen_temp = delta[mask_unseen]
    Delta_unseen_temp = Delta[mask_unseen]
    gtab_unseen.append(grad.gradient_table(bvals_unseen_temp, bvecs_unseen_temp, big_delta=Delta_unseen_temp, small_delta=delta_unseen_temp))

In [5]:
p = parallelization()

In [6]:
models = p.start(Model, n, gtab_seen, sphere=[sphere], isotropic=[BiExponentialIsotropicModel])

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 0.00min
 25%      0.00min remaining
 50%      0.01min remaining
 75%      0.01min remaining


In [7]:
fits = p.start([i.fit for i in models], n, data_seen, te_seen)

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 4.27min
 25%      0.00min remaining
 50%      0.01min remaining
 75%      0.00min remaining


In [8]:
predicts = p.start([i.predict for i in fits], n, gtab_unseen, te_unseen)

Parallelization starts on 4 CPUs.
100%      0.00min remaining
Total Time needed: 1.83min
 25%      0.01min remaining
 50%      0.01min remaining
 75%      0.00min remaining


In [9]:
LSEs = np.zeros(n)
for j in range(0,n):
    LSEs[j] = utils.LSE(predicts[j], data_unseen[j])
    print("LSE=%s"%LSEs[j])

LSE=2.93207617959
LSE=3.89976300354
LSE=2.46173546663
LSE=1.5428659537


In [10]:
print("Mean LSE = %s"%np.mean(LSEs))

Mean LSE = 2.70911015086
